# [주간] 기업의 종가와 확진자 수 상관관계

## 0. Settings

In [19]:
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import requests
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
plt.rc("font", family="Malgun Gothic")
plt.rc("axes", unicode_minus=False)
plt.style.use("fivethirtyeight")
from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

## 1. COVID19 데이터
**['Date', 'Daily_cases',	'Week',	'Day',	'Weekly_cases']**  
**날짜 / 일일 확진자 수 / 주차 / 요일 / 주차 확진자수 합계**

In [20]:
covid = pd.read_excel('practice_covid.xlsx')
covid.head()

,Date,New_cases
0,2020-01-03,0
1,2020-01-04,0
2,2020-01-05,0
3,2020-01-06,0
4,2020-01-07,0


### 1.1 Date + 'Week', 'Day' 컬럼 추가
* 이때, Week 은 해당 연도의 몇 번째 주차인지 알려주는 컬럼입니다.
* 요일은 0=월요일, 4=금요일, 6=일요일 입니다. 즉, 한 주차는 월요일~일요일 입니다. 
* covid 데이터는 주차(0~6) 별 New_cases의 합계 | stock 데이터는 금요일(4) 종가  
    이렇게 하기로 했는데, 그렇게 해도 문제 없습니다.  
    다만, merge 할 때는 주차로 merge 해야하므로, stock 데이터에도 'Week' 컬럼을 추가 합니다.  
    또한, 헷갈리지 않도록 여기에도 'Day' 컬럼을 추가해두겠습니다.

In [21]:
# Date가 해당 연도의 몇 번째 주차인지 알려주는 컬럼을 추가 합니다.
# 월~일 = 하나의 week
covid["Week"] = covid["Date"].dt.week
covid[["Date", "Week", " New_cases"]].head()

,Date,Week,New_cases
0,2020-01-03,1,0
1,2020-01-04,1,0
2,2020-01-05,1,0
3,2020-01-06,2,0
4,2020-01-07,2,0


In [22]:
# 무슨 요일인지 추출합니다.
# 0=월요일 , 4=금요일
# covid["새컬럼명"] = covid["빼오려는 날짜 컬럼"].dt.dayofweek
covid["Day"] = covid["Date"].dt.dayofweek
covid[["Date", "Week", "Day", " New_cases"]].head()

,Date,Week,Day,New_cases
0,2020-01-03,1,4,0
1,2020-01-04,1,5,0
2,2020-01-05,1,6,0
3,2020-01-06,2,0,0
4,2020-01-07,2,1,0


### 1.2 주차 별 확진자수 'Weekly_cases' 추가

In [23]:
# 주차 별 확진자 수를 구합니다.
covid_w= covid.groupby(["Week"])[" New_cases"].sum()
covid_w = pd.DataFrame(covid_w)
covid_w.columns = ['Weekly_cases']
covid_w.reset_index()
covid_w.head()

,Weekly_cases
Week,
1,0
2,0
3,5
4,2
5,12


In [24]:
# 기존 데이터에 주차 별 확진자수 컬럼을 groupby를 통해 추가합니다.
# 주의: 돌릴 때마다 새 컬럼 추가되므로 여기 손대면 처음부터 다시 돌릴 것
covid = covid.merge(covid_w, left_on="Week", right_on=covid_w.index, how="left")
covid.head()

,Date,New_cases,Week,Day,Weekly_cases
0,2020-01-03,0,1,4,0
1,2020-01-04,0,1,5,0
2,2020-01-05,0,1,6,0
3,2020-01-06,0,2,0,0
4,2020-01-07,0,2,1,0


In [25]:
# 컬럼 명을 변경합니다.
covid.columns = ["Date", "Daily_cases", "Week", "Day", "Weekly_cases"]
covid.head()

,Date,Daily_cases,Week,Day,Weekly_cases
0,2020-01-03,0,1,4,0
1,2020-01-04,0,1,5,0
2,2020-01-05,0,1,6,0
3,2020-01-06,0,2,0,0
4,2020-01-07,0,2,1,0


### 1.3 저장하기

In [31]:
# 필요할지도 모르니 다 저장
covid.to_excel("1013_covid_daily_weekly.xlsx", index=False, encoding="utf8")
covid.to_pickle("1013_covid_daily_weekly.pkl")
covid.to_html()
covid.to_html("1013_covid_daily_weekly.html", index=False)

In [32]:
# 잘 저장되었나 확인
covid = pd.read_excel('1013_covid_daily_weekly.xlsx')
covid.tail()

,Date,Daily_cases,Week,Day,Weekly_cases
275,2020-10-04,64,40,6,480
276,2020-10-05,73,41,0,331
277,2020-10-06,75,41,1,331
278,2020-10-07,114,41,2,331
279,2020-10-08,69,41,3,331


============================현재(20.10.13 21:00) 여기까지 완료 (해인)==============================

## 2. 기업 주가 정보
**['Date', 'Company', 'Close', 'Week', 'Day' 'Weekly_close']**  
**날짜 / 기업명 / 일일 종가 / 주차 / 요일 / 금요일 종가**

### 2.1 특정 기업 정보 크롤링 ['날짜', '종가']

#### 2.1.1 상장법인 목록 불러오기 ['회사명', '종목코드']

In [35]:
# 회사명으로 주식 종목 코드를 획득할 수 있도록 하는 함수
def get_code(df, name):
    code = df.query("name=='{}'".format(name))['code'].to_string(index=False)
    # 위와같이 code명을 가져오면 앞에 공백이 붙어있는 상황이 발생하여 앞뒤로 sript() 하여 공백 제거
    code = code.strip()
    return code

# excel 파일을 다운로드하는거와 동시에 pandas에 load하기
# 흔히 사용하는 df라는 변수는 data frame을 의미합니다.
company_code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download', header=0)[0]

# data frame정리
company_code_df = company_code_df[['회사명', '종목코드']]

# data frame title 변경 '회사명' = name, 종목코드 = 'code'
company_code_df = company_code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})

# 종목코드는 6자리로 구분되기때문에 0을 채워 6자리로 변경
company_code_df.code = company_code_df.code.map('{:06d}'.format)
company_code_df.head()

,name,code
0,JS전선,005560
1,거북선2호,101380
2,거북선6호,114140
3,교보메리츠,064900
4,국제관광공사,028780


#### 2.1.2 기업의 주가 정보 크롤링

In [51]:
def get_price(company_code):
    # count=439에서 3000은 과거 439 영업일간의 데이터를 의미. 사용자가 조절 가능 - 2019.01.02 ~ 2020.10.13
    url = "https://fchart.stock.naver.com/sise.nhn?symbol={}&timeframe=day&count=439&requestType=0".format(company_code)
    get_result = requests.get(url)
    bs_obj = BeautifulSoup(get_result.content, "html.parser")
    
    # information
    inf = bs_obj.select('item')
    columns = ['Date', 'Open' ,'High', 'Low', 'Close', 'Volume']
    df1_inf = pd.DataFrame([], columns = columns, index = range(len(inf)))
    
    for i in range(len(inf)):
        df1_inf.iloc[i] = str(inf[i]['data']).split('|')
    
    df1_inf.index = pd.to_datetime(df1_inf['Date'])
    
    return df1_inf.drop('Date', axis=1).astype(float)

In [52]:
samsung_price = get_price('025980') #추후에 여기 종목 코드 - 1.1.1 을 넣어서 반복 될 수 있도록 만들기
samsung_price.head()

,Open,High,Low,Close,Volume
Date,,,,,
2019-01-02,19450.0,19900.0,18550.0,19800.0,5371377.0
2019-01-03,19700.0,21200.0,19300.0,20400.0,6705977.0
2019-01-04,19950.0,21650.0,19500.0,21650.0,3791053.0
2019-01-07,21500.0,22100.0,21100.0,21900.0,3955680.0
2019-01-08,21800.0,27700.0,21750.0,26100.0,25567152.0


In [67]:
# 코딩하기 힘드니까 Date를 인덱스에서 빼줍니다.
samsung = samsung_price[["Close"]]
samsung = samsung.reset_index()
samsung.head()

,Date,Close
0,2019-01-02,19800.0
1,2019-01-03,20400.0
2,2019-01-04,21650.0
3,2019-01-07,21900.0
4,2019-01-08,26100.0


1.1.1 과 1.1.2 를 잘 섞어서 for 문으로 만들면 모든 상장 기업(3875)의 주가 정보를 얻을 수 있다.  
Date가 인덱스에 있어서 건들지 못하는 게 있는데, 최종으로는 넣더라도 지금은 컬럼으로 어떻게 빼야할듯..
(지금 상태에서 for문 만들면 복잡해져서 일단 보류)  

#### for문으로 만들면 최종으로 기대하는 모양 ['Date', 'Company', 'Close', 'Week', 'Day', 'Weekly_close']  
* 현재는 company 이름 혹은 코드가 안나오는데, 어떻게 만들면 나오게 할 수 있을듯!
* 이때, weekly_close = friday_close
* 이때, 'Week'은 코로나 데이터의 'Week'과 일치 (아마 따로 조치 안해도 자동으로 같을 것, 다만! 2019, 2020 연도가 두 개라 어찌 나올지..?)

### 2.2 Date + 'Week', 'Day' 컬럼 추가
여기에서 기대하는 모양의 컬럼 ['Date', 'Week', 'Day' 'Weekly_Close']  
* 이때, 나중에 코로나 데이터 merge를 위해서 'Week'은 금요일로 하지 않고 주차로 해야할듯
* 그러면서 Weekly Close는 금요일 종가를 써야하므로, Week 외에 별도로 Day(요일) 컬럼도 추가

In [68]:
# 해당 연도의 몇 번째 "주차"인지 추출합니다.
samsung["Week"] = samsung["Date"].dt.week
samsung[["Date", "Week", "Close"]].head()

,Date,Week,Close
0,2019-01-02,1,19800.0
1,2019-01-03,1,20400.0
2,2019-01-04,1,21650.0
3,2019-01-07,2,21900.0
4,2019-01-08,2,26100.0


In [69]:
# 요일을 추출합니다.
samsung["Day"] = samsung["Date"].dt.dayofweek
samsung.head()

,Date,Close,Week,Day
0,2019-01-02,19800.0,1,2
1,2019-01-03,20400.0,1,3
2,2019-01-04,21650.0,1,4
3,2019-01-07,21900.0,2,0
4,2019-01-08,26100.0,2,1


### 2.3 금요일 종가 컬럼 추가
* 'Weekly_close' 라고 쓰고 'Friday_close'라고 읽는다.

### 2.3 저장하기

In [ ]:
# 작업 전
tour_covid.to_excel("tour_covid.xlsx", index=False, encoding="utf8")
tour_covid.to_pickle("tour_covid.pkl")
tour_covid.to_html()
tour_covid.to_html("tour_covid.html", index=False)

## 3. 회귀분석

## 요 아래부터는 복사본이라 위와 직접 관계 없음 필요한 것만 가져다 쓰기

### 2.1 금요일 종가

In [2]:
game = pd.read_excel('game_all_close.xlsx')
game.tail()

,Date,위메이드,게임빌,컴투스,넥슨지티,펄어비스,넷마블,NHN,선데이토즈,더블유게임즈
248,2020-09-29,37300,35100,114600,15250,202200,15250,74400,21700,74500
249,2020-10-05,37000,35800,118500,15250,202300,15250,74900,22750,73200
250,2020-10-06,39150,36050,119100,15250,205300,15250,75000,22750,73100
251,2020-10-07,38550,35400,116700,15400,211800,15400,75200,22700,72800
252,2020-10-08,38100,34950,116600,15200,216000,15200,75500,22450,72500


In [3]:
covid = pd.read_excel('practice_covid.xlsx')
covid.head()

,Date,New_cases
0,2020-01-03,0
1,2020-01-04,0
2,2020-01-05,0
3,2020-01-06,0
4,2020-01-07,0


In [5]:
covid.columns = ["Date", "확진자 수"]
covid.head()

,Date,확진자 수
0,2020-01-03,0
1,2020-01-04,0
2,2020-01-05,0
3,2020-01-06,0
4,2020-01-07,0


## game 공휴일 채우기

In [7]:
# 범위를 정하기에 앞서서 첫날 과 마지막날 정의하기
game_first_day = game.iloc[0, 0]
game_last_day = game.iloc[252, 0]
game_first_day, game_last_day

(Timestamp('2019-10-01 00:00:00'), Timestamp('2020-10-08 00:00:00'))

In [8]:
# 범위 함수로 빠진 날짜까지 불러오기
game_days = pd.date_range(game_first_day, game_last_day)
game_days

DatetimeIndex(['2019-10-01', '2019-10-02', '2019-10-03', '2019-10-04',
               '2019-10-05', '2019-10-06', '2019-10-07', '2019-10-08',
               '2019-10-09', '2019-10-10',
               ...
               '2020-09-29', '2020-09-30', '2020-10-01', '2020-10-02',
               '2020-10-03', '2020-10-04', '2020-10-05', '2020-10-06',
               '2020-10-07', '2020-10-08'],
              dtype='datetime64[ns]', length=374, freq='D')

In [9]:
# 데이터프레임으로 넣어주기
df_game_days = pd.DataFrame({"날짜": game_days})
df_game_days.head()

,날짜
0,2019-10-01
1,2019-10-02
2,2019-10-03
3,2019-10-04
4,2019-10-05


In [10]:
# 합치기
game_all_day = df_game_days.merge(game, left_on="날짜", right_on=game["Date"], how="left")
game_all_day[:5]

,날짜,Date,위메이드,게임빌,컴투스,넥슨지티,펄어비스,넷마블,NHN,선데이토즈,더블유게임즈
0,2019-10-01,2019-10-01,30950.0,31000.0,94500.0,7170.0,200900.0,7170.0,63300.0,15850.0,56100.0
1,2019-10-02,2019-10-02,30000.0,30300.0,94800.0,7140.0,205400.0,7140.0,62900.0,15650.0,55800.0
2,2019-10-03,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-10-04,2019-10-04,29750.0,30050.0,93500.0,7070.0,203000.0,7070.0,62400.0,18800.0,54000.0
4,2019-10-05,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
game_all_day[['위메이드', '게임빌', '컴투스', '넥슨지티', '펄어비스', '넷마블', 'NHN', '선데이토즈',
       '더블유게임즈']] = game_all_day[['위메이드', '게임빌', '컴투스', '넥슨지티', '펄어비스', '넷마블', 'NHN', '선데이토즈',
       '더블유게임즈']].fillna(method="bfill")
game_all_day[:5]

,날짜,Date,위메이드,게임빌,컴투스,넥슨지티,펄어비스,넷마블,NHN,선데이토즈,더블유게임즈
0,2019-10-01,2019-10-01,30950.0,31000.0,94500.0,7170.0,200900.0,7170.0,63300.0,15850.0,56100.0
1,2019-10-02,2019-10-02,30000.0,30300.0,94800.0,7140.0,205400.0,7140.0,62900.0,15650.0,55800.0
2,2019-10-03,NaT,29750.0,30050.0,93500.0,7070.0,203000.0,7070.0,62400.0,18800.0,54000.0
3,2019-10-04,2019-10-04,29750.0,30050.0,93500.0,7070.0,203000.0,7070.0,62400.0,18800.0,54000.0
4,2019-10-05,NaT,30000.0,29850.0,93800.0,6980.0,199000.0,6980.0,61500.0,19550.0,53000.0


In [24]:
game.columns

Index(['날짜', '위메이드', '게임빌', '컴투스', '넥슨지티', '펄어비스', '넷마블', 'NHN', '선데이토즈',
       '더블유게임즈'],
      dtype='object')

In [25]:
game_all_day[['Date', '위메이드', '게임빌', '컴투스', '넥슨지티', '펄어비스', '넷마블', 'NHN', '선데이토즈',
       '더블유게임즈']] = game_all_day[['Date', '위메이드', '게임빌', '컴투스', '넥슨지티', '펄어비스', '넷마블', 'NHN', '선데이토즈',
       '더블유게임즈']]
game_all_day.head()

,날짜,Date,위메이드,게임빌,컴투스,넥슨지티,펄어비스,넷마블,NHN,선데이토즈,더블유게임즈
0,2019-10-01,2019-10-01,30950.0,31000.0,94500.0,7170.0,200900.0,7170.0,63300.0,15850.0,56100.0
1,2019-10-02,2019-10-02,30000.0,30300.0,94800.0,7140.0,205400.0,7140.0,62900.0,15650.0,55800.0
2,2019-10-03,NaT,29750.0,30050.0,93500.0,7070.0,203000.0,7070.0,62400.0,18800.0,54000.0
3,2019-10-04,2019-10-04,29750.0,30050.0,93500.0,7070.0,203000.0,7070.0,62400.0,18800.0,54000.0
4,2019-10-05,NaT,30000.0,29850.0,93800.0,6980.0,199000.0,6980.0,61500.0,19550.0,53000.0


In [26]:
game = game_all_day[['날짜', '위메이드', '게임빌', '컴투스', '넥슨지티', '펄어비스', '넷마블', 'NHN', '선데이토즈',
       '더블유게임즈']]

## game + covid Merge

In [27]:
game_covid = game.merge(covid, left_on="날짜", right_on=covid["Date"], how="left")
game_covid.head()

,날짜,위메이드,게임빌,컴투스,넥슨지티,펄어비스,넷마블,NHN,선데이토즈,더블유게임즈,Date,확진자 수
0,2019-10-01,30950.0,31000.0,94500.0,7170.0,200900.0,7170.0,63300.0,15850.0,56100.0,NaT,NaN
1,2019-10-02,30000.0,30300.0,94800.0,7140.0,205400.0,7140.0,62900.0,15650.0,55800.0,NaT,NaN
2,2019-10-03,29750.0,30050.0,93500.0,7070.0,203000.0,7070.0,62400.0,18800.0,54000.0,NaT,NaN
3,2019-10-04,29750.0,30050.0,93500.0,7070.0,203000.0,7070.0,62400.0,18800.0,54000.0,NaT,NaN
4,2019-10-05,30000.0,29850.0,93800.0,6980.0,199000.0,6980.0,61500.0,19550.0,53000.0,NaT,NaN


In [28]:
game_covid.columns

Index(['날짜', '위메이드', '게임빌', '컴투스', '넥슨지티', '펄어비스', '넷마블', 'NHN', '선데이토즈',
       '더블유게임즈', 'Date', '확진자 수'],
      dtype='object')

In [29]:
# 코로나 데이터 중 Date 날리기
game_covid = game_covid[['날짜', '위메이드', '게임빌', '컴투스', '넥슨지티', '펄어비스', '넷마블', 'NHN', '선데이토즈',
       '더블유게임즈', '확진자 수']]

In [30]:
game_covid["확진자 수"] = game_covid["확진자 수"].fillna(0)
game_covid.head()

,날짜,위메이드,게임빌,컴투스,넥슨지티,펄어비스,넷마블,NHN,선데이토즈,더블유게임즈,확진자 수
0,2019-10-01,30950.0,31000.0,94500.0,7170.0,200900.0,7170.0,63300.0,15850.0,56100.0,0.0
1,2019-10-02,30000.0,30300.0,94800.0,7140.0,205400.0,7140.0,62900.0,15650.0,55800.0,0.0
2,2019-10-03,29750.0,30050.0,93500.0,7070.0,203000.0,7070.0,62400.0,18800.0,54000.0,0.0
3,2019-10-04,29750.0,30050.0,93500.0,7070.0,203000.0,7070.0,62400.0,18800.0,54000.0,0.0
4,2019-10-05,30000.0,29850.0,93800.0,6980.0,199000.0,6980.0,61500.0,19550.0,53000.0,0.0


* 저장하기

In [31]:
game_covid.to_excel("game_covid.xlsx", index=False, encoding="utf8")
game_covid.to_pickle("game_covid.pkl")
game_covid.to_html()
game_covid.to_html("game_covid.html", index=False)